# Exaact diagonalization study of Heisenberg chain by Lanczos algorithm

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [4]:
# set parameters
N=13
dimension=2**N
#spin chain with all spin-down
z='0'*N
# initialize hamiltonian
H=np.zeros((dimension,dimension))
z

'0000000000000'

In [5]:
# Matrix Construction
for a in range(dimension):
    state_chain=bin(a)[2:] # the first two should be omitted for this 'bin' function
    l=len(state_chain)
    state_chain=z[0:N-l]+state_chain # make the length equal to N
    #print(state_chain)
  # for PBC, we set i in range(N)
  # for OBC, we set i in range(N-1)
    for i in range(N):
        j=np.mod(i+1,N)
 #       print(state_chain)
        if state_chain[i]==state_chain[j]: # i=j only diagonal elements
            H[a,a]+=0.25
#            print('a:',a)
        else:                              # else, the raising/lowering operators also have contributions
            H[a,a]-=0.25
            # then exchange i,j
            element_i=state_chain[i]
            element_j=state_chain[j]
            #flip
            if j==0:
#here we are doing the concatenation of string (you can try other methods)
#                    print(state_chain)
                state_chain1=element_i+state_chain[1:N-1]+element_j
            else:
                state_chain1=state_chain[0:i]+element_j+element_i+state_chain[j+1:]
#            print(state_chain)
            b=int(state_chain1,2)
#            print('a:',a)
#            print('b:',b)
            H[a,b]+=0.5

# Lanczos
https://en.wikipedia.org/wiki/Lanczos_algorithm

In [ ]:
def random_orthogonal(V,j):
    N = V.shape[0]
    v = np.random.randn(N)+1j*np.random.randn(N)
    for i in range(j):
        u = V[:,i]
        v -= np.vdot(u,v)*u/np.vdot(u,u)
    v = v/np.linalg.norm(v)
    return v

In [ ]:
def Lanczos(Hamiltonian):
    H = np.copy(Hamiltonian)
    N = H.shape[0]
    m = 100
    H = 0.5*(H + H.conj().T)  # ensure Hermitian
    V_matrix = np.zeros((N, m), dtype=complex)
    T = np.zeros((m, m), dtype=complex)

    # # scale-aware tolerance
    # eps = np.finfo(float).eps
    # Hnorm = np.linalg.norm(H, ord=2)
    # tol = np.sqrt(eps) * (Hnorm if Hnorm>0 else 1.0)

    # init
    v = np.random.randn(N) + 1j*np.random.randn(N)
    v /= np.linalg.norm(v)
    V_matrix[:, 0] = v

    w_prime = H @ v
    alpha = np.vdot(v, w_prime)
    w = w_prime - alpha * v
    T[0, 0] = alpha

    for i in range(1, m):
        if (i+1)%(m//10) == 0:
            percent = int((i+1)/m*100)
            print(f"Lanczos Progress:{percent}%")
        #print(i)
        h = V_matrix[:, :i].conj().T @ w
        w -= V_matrix[:, :i] @ h
        beta = np.linalg.norm(w)
        v_old = v
        v = w / beta
        if beta < 10e-6:
            v = random_orthogonal(V_matrix,i)
        else:
            v = w / beta
        V_matrix[:, i] = v
        w_prime = H @ v - beta * v_old
        alpha = np.vdot(v, w_prime)
        w = w_prime - alpha * v

        T[i, i] = alpha
        T[i, i-1] = beta
        T[i-1, i] = beta

    return T




# Implicitly Shifted QR Decomposition Method to solve the eigenvalues of tridiagonal matrix

Algorithm 4 from the book: [Handbook of Linear Algebra by Hogben](https://math.ecnu.edu.cn/~jypan/Teaching/MC/refs/2014%20Symmetric%20Matrix%20Eigenvalue%20Techniques.pdf)




In [6]:
# import numpy as np

# def generate_symmetric_tridiagonal_matrix(n):

#     main_diag = np.random.rand(n)
#     off_diag = np.random.rand(n-1)
#     T = np.zeros((n, n))
#     np.fill_diagonal(T, main_diag)
#     np.fill_diagonal(T[1:], off_diag)
#     np.fill_diagonal(T[:, 1:], off_diag)

#     return T


In [7]:
def Givens(x,y):
    G = np.zeros([2,2])
    if y == 0:
        c = 1
        s = 0
    elif x == 0:
        c = 0
        s = np.sign(y)
    else:
        c = abs(x)/math.sqrt(abs(x)**2+abs(y)**2)
        s = x/abs(x)*y/math.sqrt(abs(x)**2+abs(y)**2)
    G[0,0] = c
    G[1,1] = c
    G[0,1] = s
    G[1,0] = -s
    return G

def QR_iteration(T_matrix):
    T = np.copy(T_matrix)
    T = np.real(T)
    epsilon = 10e-10
    if T.shape == (1,1):
        return T
    n_itr = 0

    while True:
        n_itr += 1
        N = T.shape[0]
        tau = (T[N-2,N-2]-T[N-1,N-1])/2
        mu = T[N-1,N-1]-T[N-1,N-2]**2/(tau+np.sign(tau)*np.sqrt(tau**2+T[N-1,N-2]**2))

        for i in range(0,N-1):
            if i==0:
                G = Givens(T[0,0]-mu,T[1,0])
            else:
                G = Givens(T[i,i-1],T[i+1,i-1])

            iL = max(i-1,0)
            iR = min(N,i+3)

            T[i:i+2,iL:iR] = G@T[i:i+2,iL:iR]
            T[iL:iR,i:i+2] = T[iL:iR,i:i+2]@G.T

            if abs(T[i,i+1])**2 <= epsilon**2*abs(T[i,i]*T[i+1,i+1]):
                #print(n_itr)
                T[i,i+1] = T[i+1,i] = 0.0
                T[0:i+1,0:i+1] = QR_iteration(T[0:i+1,0:i+1])
                T[i+1:N,i+1:N] = QR_iteration(T[i+1:N,i+1:N])
                return T



In [8]:
# N_ = 1000
# A = np.random.randn(N_, N_) + 1j * np.random.randn(N_, N_)
# H = (A + A.conj().T) / 2

In [ ]:
eig_value_H=np.real(np.linalg.eig(H)[0])
eig_value_H = np.sort(eig_value_H)
print(eig_value_H)

[-5.62958433 -5.62958433 -5.62958433 ...  3.25        3.25
  3.25      ]


In [ ]:
T = Lanczos(H)
Lambda = QR_iteration(T)
eigs_lan = np.diag(Lambda)
eigs_lan = np.sort(eigs_lan)
print(eigs_lan)

Lanczos Progress:10%
Lanczos Progress:20%
Lanczos Progress:30%
Lanczos Progress:40%
Lanczos Progress:50%
Lanczos Progress:60%
Lanczos Progress:70%
Lanczos Progress:80%
Lanczos Progress:90%
Lanczos Progress:100%
[-5.62958433e+00 -5.62958433e+00 -5.00187552e+00 -5.00187543e+00
 -4.92049447e+00 -4.91599812e+00 -4.65676458e+00 -4.41833955e+00
 -4.40146825e+00 -4.35673429e+00 -4.24574167e+00 -4.23715427e+00
 -4.19383491e+00 -4.06506892e+00 -4.06146668e+00 -3.93997906e+00
 -3.87610964e+00 -3.84726584e+00 -3.77203089e+00 -3.74228803e+00
 -3.70440401e+00 -3.62132376e+00 -3.56221464e+00 -3.46159701e+00
 -3.40340101e+00 -3.31880072e+00 -3.25987643e+00 -3.13013292e+00
 -3.05731735e+00 -2.95634787e+00 -2.87398493e+00 -2.78372290e+00
 -2.67499168e+00 -2.58195434e+00 -2.46809723e+00 -2.37336138e+00
 -2.26500048e+00 -2.14325699e+00 -2.06220416e+00 -1.93785925e+00
 -1.83693430e+00 -1.71943491e+00 -1.60859033e+00 -1.50286732e+00
 -1.37540867e+00 -1.26089480e+00 -1.14113485e+00 -1.03321166e+00
 -9.02173